In [1]:
from werpy_weighted import werpy
import whisper
from whisper.tokenizer import get_tokenizer

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd
from tqdm.notebook import tqdm

# v6

In [4]:
tokenizer = get_tokenizer(multilingual=False)  # use multilingual=True if using multilingual model
number_tokens = [
    i 
    for i in range(tokenizer.eot)
    if all(c in "0123456789" for c in tokenizer.decode([i]).removeprefix(" "))
]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

whisper_model = whisper.load_model('base.en').to(device)

In [6]:
split_txt_train_path = r'.\preprocessed_data\Ego4D_final_v6\train.txt'
train_uids = []
gt_transcripts_dict = {}
with open(split_txt_train_path) as file:
    for line in file:
        # print(line.split('|')[0])
        train_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [7]:
len(train_uids)

27292

In [8]:
gt_folder = r'.\raw_data\Ego4D_final_v6\train\Ego4D_final_v6'

In [9]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(train_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(gt_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/27292 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
whisper_df_v6gt_train = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

ValueError: All arrays must be of the same length

In [ ]:
whisper_df_v6gt_train.head()

In [ ]:
whisper_df_v6gt_train.to_csv(r".\jupyter_walkthrough\metrics\whisper_df_v6gt_train.csv", index=False)

In [6]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
gt_transcripts_dict = {}
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [7]:
len(val_uids)

2772

In [8]:
gt_folder = r'.\raw_data\Ego4D_final_v6\val\Ego4D_final_v6'

In [9]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(val_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(gt_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/2772 [00:00<?, ?it/s]

CPU times: total: 1h 2min 17s
Wall time: 25min 25s


In [10]:
whisper_df_v6gt_val = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

In [11]:
whisper_df_v6gt_val.head()

,uid,whisper_transcripts,gt_transcripts
0,9d58583c-20de-439d-b1cd-9c2265bdedd8,What?,what \n
1,4a506707-52ea-493e-98c4-f667e3222d44,"Pretty shit, to be honest.",pretty shit to be honest \n
2,0b67f942-6ebf-403d-aff0-f4e5d62d3140,One of them together.,oh when you nail them together they won't be ...
3,c237c6d5-f413-4e25-82e2-af7b408d390a,"No, no, I'm certainly. As if there were right...",no not necessary cuz if they're warped right ...
4,b8ed5cf9-b8ff-4da3-a764-a59020b85277,See the dots and there.,or we could just use the \n


In [12]:
whisper_df_v6gt_val.to_csv(r".\jupyter_walkthrough\metrics\whisper_df_v6gt_val.csv", index=False)

In [38]:
import pandas as pd
whisper_df_v5gt = pd.read_csv(r".\jupyter_walkthrough\metrics\whisper_df_v5gt.csv")

In [39]:
whisper_df_v5gt = whisper_df_v5gt.dropna()

In [40]:
normalized_ref = werpy.normalize(
    whisper_df_v5gt['gt_transcripts'].values.tolist(),
)
normalized_hypo = werpy.normalize(
    whisper_df_v5gt['whisper_transcripts'].values.tolist(),
)

In [41]:
normalized_ref[0]

'what'

In [42]:
normalized_hypo[0]

'whats'

In [43]:
%%time
summary = werpy.summary(normalized_ref, normalized_hypo)
summary

CPU times: total: 141 ms
Wall time: 189 ms


,wer,ld,m,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,4.000000,4,1,0,0,1,[],[],"[(what, whats)]"
1,0.000000,0,5,0,0,0,[],[],[]
2,3.083333,37,12,0,7,4,[],"[oh, they, wont, be, as, hard, right]","[(when, good), (you, morning), (nail, and), (t..."
3,0.611111,11,18,0,1,2,[],[no],"[(necessary, necessarily), (cuz, because)]"
4,3.800000,19,5,0,1,4,[],[stand],"[(them, that), (all, can), (down, reflect), (f..."
...,...,...,...,...,...,...,...,...,...
2665,0.166667,7,42,1,0,1,[player],[],"[(a, the)]"
2666,2.960784,151,51,0,49,1,[],"[id, read, the, instructions, to, you, right, ...","[(first, like)]"
2667,0.339286,19,56,0,5,1,[],"[a, and, so, on, and]","[(then, when)]"
2668,0.750000,30,40,10,0,0,"[dukkid, dont, accept, ahh, dukk, nikr, nikr, ...",[],[]


In [44]:
I = summary['insertions'].sum()
D = summary['deletions'].sum()
S = summary['substitutions'].sum()
M = summary['m'].sum()

In [45]:
summary.to_csv(r".\jupyter_walkthrough\metrics\v5gt_wer_summary.csv", index=False)

In [46]:
wer = (I + D + S) / M

In [47]:
print(f"wer: {wer}")

wer: 0.6610960757780785


In [48]:
print(f"I, D, S, M: {I, D, S, M}")

I, D, S, M: (2875, 2097, 4799, 14780)


In [49]:
print(f"I/M, D/M, S/M: {I/M, D/M, S/M}")

I/M, D/M, S/M: (0.19451962110960758, 0.1418809201623816, 0.3246955345060893)
